In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.optimize import minimize
pd.set_option('display.max_columns', None) # Show all columns

In [2]:
# Downloading the Adjusted Close Prices of Nifty Commodities Index Stocks
raw = pd.read_csv('niftycomm.csv')
tickers = [ticker + '.NS' for ticker in raw['Symbol'].tolist()]
end_date = pd.to_datetime('2024-05-15')
start_date = end_date - pd.DateOffset(years=3)
df = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

[*********************100%%**********************]  30 of 30 completed


In [3]:
df = df.drop(columns = ['ADANIENSOL.NS']) # Dropping ADANIENSOL.NS as it has very few data points
df = df.ffill() # Filling missing values with the previous day's value

In [4]:
#df.to_csv('niftycomm_adj_close_prices.csv')

In [5]:
# Calculating lognormal daily returns for all the stocks
daily_returns = np.log(df/df.shift(1)).dropna() # Dropping the first row as it will have NaN value
tickers = daily_returns.columns # Updating the tickers list as we have dropped ADANIENSOL.NS
daily_returns.to_csv('niftycomm_daily_returns.csv')
daily_returns.head()

,ACC.NS,ADANIGREEN.NS,ADANIPOWER.NS,AMBUJACEM.NS,APLAPOLLO.NS,BPCL.NS,COALINDIA.NS,DALBHARAT.NS,DEEPAKNTR.NS,GRASIM.NS,HINDALCO.NS,HINDPETRO.NS,IOC.NS,JINDALSTEL.NS,JSWSTEEL.NS,NTPC.NS,ONGC.NS,PIDILITIND.NS,PIIND.NS,RELIANCE.NS,SAIL.NS,SHREECEM.NS,SRF.NS,TATACHEM.NS,TATAPOWER.NS,TATASTEEL.NS,ULTRACEMCO.NS,UPL.NS,VEDL.NS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-05-18,0.018915,0.048104,0.048766,0.017417,-0.000279,0.010761,-0.011541,0.021507,0.043076,-0.001041,0.023018,0.050896,0.034212,0.008915,0.004260,0.004918,0.016063,0.001083,0.015105,0.013981,0.010667,0.005345,0.009489,0.014104,0.018200,0.025014,0.018301,-0.001753,0.024285
2021-05-19,-0.006568,0.036771,0.004895,0.006316,0.026226,-0.004470,0.036209,-0.011990,-0.010589,-0.001941,0.002980,0.028826,0.015511,-0.040760,-0.016301,-0.001339,-0.010390,0.014578,-0.064093,0.004567,-0.025716,0.002109,-0.002087,-0.002313,-0.010496,-0.013437,-0.013894,0.016436,-0.009856
2021-05-20,0.015906,0.020060,-0.022717,0.007371,-0.016947,0.018530,-0.034844,0.010971,-0.000676,-0.010452,-0.045918,-0.023557,-0.023597,-0.050564,-0.016211,-0.001788,-0.027351,0.007830,0.005913,-0.005851,-0.057204,-0.000812,0.003572,-0.007680,-0.012548,-0.052187,0.001926,-0.018908,-0.024710
2021-05-21,0.003619,0.023638,0.009940,0.004521,-0.010874,0.013649,0.003743,0.008942,-0.006815,-0.002475,0.011871,0.036488,-0.003828,0.005223,0.005313,0.009795,0.008461,0.021436,-0.008191,0.008601,0.004518,0.004074,0.002312,0.001272,0.018764,0.007258,0.009651,0.015963,-0.007826
2021-05-24,0.009722,0.046434,-0.000495,0.014669,0.000199,0.025272,0.011483,0.025639,0.014088,-0.000802,-0.001284,0.029422,0.048196,-0.013234,-0.024283,0.000443,0.002657,0.040571,-0.006315,-0.008475,0.022291,-0.026162,-0.006570,-0.000848,0.022621,-0.019138,-0.011394,0.012108,0.004739


In [6]:
# Creating the covariance matrix
cov_mat = daily_returns.cov()
cov_mat.to_csv('niftycomm_cov_matrix.csv')
cov_mat

,ACC.NS,ADANIGREEN.NS,ADANIPOWER.NS,AMBUJACEM.NS,APLAPOLLO.NS,BPCL.NS,COALINDIA.NS,DALBHARAT.NS,DEEPAKNTR.NS,GRASIM.NS,HINDALCO.NS,HINDPETRO.NS,IOC.NS,JINDALSTEL.NS,JSWSTEEL.NS,NTPC.NS,ONGC.NS,PIDILITIND.NS,PIIND.NS,RELIANCE.NS,SAIL.NS,SHREECEM.NS,SRF.NS,TATACHEM.NS,TATAPOWER.NS,TATASTEEL.NS,ULTRACEMCO.NS,UPL.NS,VEDL.NS
ACC.NS,0.000348,0.000203,0.000221,0.000339,0.000078,0.000097,0.000076,0.000184,0.000114,0.000133,0.000120,0.000112,0.000093,0.000161,0.000109,0.000087,0.000041,0.000067,0.000082,0.000076,0.000178,0.000133,0.000113,0.000120,0.000147,0.000126,0.000130,0.000095,0.000122
ADANIGREEN.NS,0.000203,0.001137,0.000533,0.000269,0.000058,0.000119,0.000104,0.000131,0.000097,0.000105,0.000142,0.000168,0.000122,0.000153,0.000139,0.000086,0.000114,0.000053,0.000081,0.000088,0.000199,0.000081,0.000107,0.000139,0.000173,0.000153,0.000065,0.000102,0.000162
ADANIPOWER.NS,0.000221,0.000533,0.001172,0.000237,0.000129,0.000120,0.000160,0.000172,0.000172,0.000132,0.000176,0.000140,0.000127,0.000221,0.000152,0.000164,0.000113,0.000071,0.000146,0.000108,0.000242,0.000128,0.000175,0.000195,0.000304,0.000174,0.000099,0.000132,0.000216
AMBUJACEM.NS,0.000339,0.000269,0.000237,0.000498,0.000077,0.000102,0.000080,0.000203,0.000120,0.000139,0.000108,0.000122,0.000109,0.000141,0.000107,0.000094,0.000045,0.000070,0.000086,0.000083,0.000177,0.000148,0.000110,0.000136,0.000166,0.000119,0.000141,0.000098,0.000140
APLAPOLLO.NS,0.000078,0.000058,0.000129,0.000077,0.000544,0.000071,0.000068,0.000102,0.000153,0.000071,0.000116,0.000086,0.000075,0.000156,0.000102,0.000055,0.000050,0.000069,0.000086,0.000068,0.000167,0.000068,0.000096,0.000106,0.000123,0.000133,0.000059,0.000078,0.000095
BPCL.NS,0.000097,0.000119,0.000120,0.000102,0.000071,0.000286,0.000120,0.000087,0.000105,0.000089,0.000089,0.000286,0.000213,0.000114,0.000091,0.000096,0.000099,0.000049,0.000079,0.000077,0.000180,0.000064,0.000081,0.000103,0.000137,0.000106,0.000069,0.000082,0.000105
COALINDIA.NS,0.000076,0.000104,0.000160,0.000080,0.000068,0.000120,0.000363,0.000080,0.000113,0.000082,0.000180,0.000138,0.000145,0.000194,0.000150,0.000147,0.000177,0.000029,0.000077,0.000098,0.000241,0.000060,0.000107,0.000152,0.000198,0.000194,0.000039,0.000116,0.000210
DALBHARAT.NS,0.000184,0.000131,0.000172,0.000203,0.000102,0.000087,0.000080,0.000412,0.000167,0.000152,0.000136,0.000089,0.000088,0.000184,0.000126,0.000078,0.000037,0.000092,0.000121,0.000072,0.000186,0.000144,0.000141,0.000136,0.000144,0.000151,0.000156,0.000122,0.000147
DEEPAKNTR.NS,0.000114,0.000097,0.000172,0.000120,0.000153,0.000105,0.000113,0.000167,0.000446,0.000109,0.000130,0.000106,0.000100,0.000182,0.000124,0.000089,0.000077,0.000100,0.000161,0.000091,0.000206,0.000083,0.000218,0.000223,0.000173,0.000154,0.000096,0.000143,0.000160
GRASIM.NS,0.000133,0.000105,0.000132,0.000139,0.000071,0.000089,0.000082,0.000152,0.000109,0.000225,0.000132,0.000093,0.000080,0.000150,0.000133,0.000073,0.000051,0.000073,0.000089,0.000077,0.000156,0.000113,0.000110,0.000117,0.000123,0.000129,0.000129,0.000090,0.000112


In [7]:
def expected_return(weights, daily_returns): # Function to calculate the returns
    return np.sum(daily_returns.mean() * weights) * 252 # Annualizing the returns (252 trading days in a year)

def expected_volatility(weights, cov_matrix): # Function to calculate the volatility
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252) # Annualizing the volatility

def neg_sharpe(weights, cov_matrix, daily_returns, risk_free_rate): # Function to calculate the negative Sharpe ratio
    return -((expected_return(weights, daily_returns) - risk_free_rate) / expected_volatility(weights, cov_matrix))

In [8]:
risk_free_rate = 0.0425 # Risk-free rate taken from average long term US treasury bond yield, usually considered risk-free

In [9]:
initial_weights = np.array([1/len(tickers)] * len(tickers)) # Initial weights for the optimization
bounds = [(0, 1)] * len(tickers) # Bounds for the weights, 0 implies no shorting allowed
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}) # Constraints for the weights, that they should sum to 1

# Optimizing the portfolio
optimal_weights = minimize(expected_volatility, initial_weights, args=(cov_mat), method='SLSQP', bounds=bounds, constraints=constraints)

In [10]:
optimal_weights.x

array([0.00000000e+00, 3.20018372e-03, 7.21906671e-17, 0.00000000e+00,
       3.60838618e-02, 4.68059780e-02, 1.06746011e-17, 1.68116576e-18,
       1.05964651e-17, 7.96551378e-18, 4.23494080e-17, 0.00000000e+00,
       0.00000000e+00, 2.32471541e-18, 0.00000000e+00, 1.03580244e-01,
       1.15787992e-01, 2.53919396e-01, 5.23349132e-02, 1.30468379e-01,
       2.15521544e-18, 7.88381422e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.44537930e-01, 3.44429811e-02,
       1.37365434e-17])

In [11]:
# Analysing the optimal portfolio
print("Portfolio Analysis\n")
exp_return = expected_return(optimal_weights.x, daily_returns)
exp_volatility = expected_volatility(optimal_weights.x, cov_mat)
sharpe_ratio = (exp_return - risk_free_rate) / exp_volatility
print(f"Expected Return: {exp_return:.2%}")
print(f"Expected Volatility: {exp_volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Portfolio Analysis

Expected Return: 18.89%
Expected Volatility: 14.37%
Sharpe Ratio: 1.02


In [12]:
# Creating a dataframe for the optimal portfolio
optimal_portfolio = pd.DataFrame(index=tickers, data={'Optimal Weights': optimal_weights.x})
optimal_portfolio.to_csv('optimal_portfolio.csv') # Saving the data to a csv file

In [13]:
# CGetting the Original Portfolio Weights
og_weights = raw['Weightage'].tolist()
og_weights.pop(1)
og_weights = np.array(og_weights)
og_weights

array([0.0075, 0.0462, 0.0395, 0.0197, 0.0071, 0.0214, 0.0451, 0.0054,
       0.0055, 0.0258, 0.0237, 0.0114, 0.0367, 0.0163, 0.0351, 0.0562,
       0.056 , 0.0247, 0.0091, 0.3132, 0.0111, 0.0153, 0.011 , 0.0044,
       0.0224, 0.0336, 0.0455, 0.0062, 0.0447])

In [14]:
# Analysing the Original Portfolio
# Analysing the optimal portfolio
print("Original Portfolio Analysis\n")
exp_return = expected_return(og_weights, daily_returns)
exp_volatility = expected_volatility(og_weights, cov_mat)
sharpe_ratio = (exp_return - risk_free_rate) / exp_volatility
print(f"Expected Return: {exp_return:.2%}")
print(f"Expected Volatility: {exp_volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Original Portfolio Analysis

Expected Return: 23.93%
Expected Volatility: 18.01%
Sharpe Ratio: 1.09
